# MoMo Talent 2024

## Prepare data

In [424]:
import pandas as pd

In [425]:
source = './data.xlsx'

In [426]:
xls = pd.ExcelFile(source)

In [427]:
for sheet_name in xls.sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(xls, sheet_name=sheet_name)
    # Save the DataFrame to a CSV file
    csv_file = f"{sheet_name.replace(' ', '_')}.csv"
    df.to_csv(csv_file, index=False)
    print(f"Exported {sheet_name.replace(' ', '_')} to {csv_file}")

Exported Questions to Questions.csv
Exported Data_Transactions to Data_Transactions.csv
Exported Data_Commission to Data_Commission.csv
Exported Data_User_Info to Data_User_Info.csv


In [428]:
transaction_df = pd.read_csv('./Data_Transactions.csv')
commission_df = pd.read_csv('./Data_Commission.csv')
user_info_df = pd.read_csv('./Data_User_Info.csv')

## Understand the data

### transaction

In [429]:
transaction_df.head()

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status
0,21269588,4169517626,2020-01-01,"10,000",13,NaN
1,28097592,4170276686,2020-01-01,"20,000",13,NaN
2,47435144,4166729310,2020-01-01,"10,000",12,NaN
3,29080935,4174460303,2020-01-01,"10,000",13,NaN
4,14591075,4168216749,2020-01-01,"10,000",12,NaN


In [430]:
transaction_df.order_id.nunique()

13495

In [431]:
transaction_df.shape

(13495, 6)

In [432]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          13495 non-null  int64 
 1   order_id         13495 non-null  int64 
 2   Date             13495 non-null  object
 3   Amount           13495 non-null  object
 4   Merchant_id      13495 non-null  int64 
 5   Purchase_status  2235 non-null   object
dtypes: int64(3), object(3)
memory usage: 632.7+ KB


In [433]:
transaction_df.dtypes

user_id             int64
order_id            int64
Date               object
Amount             object
Merchant_id         int64
Purchase_status    object
dtype: object

In [434]:
transaction_df.columns

Index(['user_id', 'order_id', 'Date', 'Amount', 'Merchant_id',
       'Purchase_status'],
      dtype='object')

In [435]:
purchase_status_distinct = transaction_df['Purchase_status'].unique()
purchase_status_distinct

array([nan, 'Mua hộ'], dtype=object)

### commission

In [436]:
commission_df.head()

,Merchant_name,Merchant_id,Rate_pct
0,Viettel,12,2
1,Mobifone,13,3
2,Vinaphone,14,4
3,Vietnamobile,15,4
4,Gmobile,16,4


In [437]:
commission_df.shape

(5, 3)

In [438]:
commission_df.dtypes

Merchant_name    object
Merchant_id       int64
Rate_pct          int64
dtype: object

In [439]:
commission_df.columns

Index(['Merchant_name', 'Merchant_id', 'Rate_pct'], dtype='object')

### user_info

In [440]:
user_info_df

,User_id,First_tran_date,Location,Age,Gender
0,41654498,2018-05-02,HCMC,33_to_37,MALE
1,51276281,2019-12-08,Other Cities,unknown,MALE
2,49152375,2019-12-20,Other Cities,23_to_27,MALE
3,5971050,2019-08-01,HCMC,28_to_32,MALE
4,48134464,2019-08-23,Other Cities,18_to_22,FEMALE
...,...,...,...,...,...
13423,39810630,2018-03-19,Other Cities,28_to_32,Nam
13424,47628375,2019-09-05,HCMC,28_to_32,female
13425,46334970,2019-04-23,Other Cities,unknown,male
13426,50760533,2019-11-13,Other Cities,18_to_22,Nữ


In [441]:
user_info_df.User_id.nunique()

13390

In [442]:
# Deduplicate user
user_info_df = user_info_df.drop_duplicates(subset='User_id')

In [443]:
user_info_df.shape

(13390, 5)

In [444]:
user_info_df.dtypes

User_id             int64
First_tran_date    object
Location           object
Age                object
Gender             object
dtype: object

In [445]:
user_info_df.columns

Index(['User_id', 'First_tran_date', 'Location', 'Age', 'Gender'], dtype='object')

In [446]:
location_distinct = user_info_df['Location'].unique()
location_distinct

array(['HCMC', 'Other Cities', 'Other', 'HN', 'Unknown',
       'Ho Chi Minh City'], dtype=object)

In [447]:
age_distinct = user_info_df['Age'].unique()
age_distinct

array(['33_to_37', 'unknown', '23_to_27', '28_to_32', '18_to_22', '>37'],
      dtype=object)

In [448]:
gender_distinct = user_info_df['Gender'].unique()
gender_distinct

array(['MALE', 'FEMALE', 'Nữ', 'M', 'female', 'Nam', 'f', 'male'],
      dtype=object)

+ The location field needs to be standardized: HCMC and Ho Chi Minh City should be considered as the same location. Similarly, Other Cities and Other are the same.
+ The gender field also needs to be standardized: FEMALE, Female, M, and female should be converted to Female, with all other entries being standardized to Male.

## Merge 3 dataframes

In [449]:
transaction_commission = pd.merge(transaction_df, commission_df, on='Merchant_id', how='left')
transaction_commission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          13495 non-null  int64 
 1   order_id         13495 non-null  int64 
 2   Date             13495 non-null  object
 3   Amount           13495 non-null  object
 4   Merchant_id      13495 non-null  int64 
 5   Purchase_status  2235 non-null   object
 6   Merchant_name    13495 non-null  object
 7   Rate_pct         13495 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 843.6+ KB


#### Standardize Purchase_Status

In [450]:
merged_df = pd.merge(transaction_commission, user_info_df, left_on = 'user_id', right_on='User_id', how='left')
merged_df['Purchase_status'] = merged_df['Purchase_status'].apply(lambda x: 1 if x == 'Mua hộ' else 0)
merged_df

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,Merchant_name,Rate_pct,User_id,First_tran_date,Location,Age,Gender
0,21269588,4169517626,2020-01-01,"10,000",13,0,Mobifone,3,21269588,2018-12-11,HN,>37,FEMALE
1,28097592,4170276686,2020-01-01,"20,000",13,0,Mobifone,3,28097592,2019-12-30,HN,>37,FEMALE
2,47435144,4166729310,2020-01-01,"10,000",12,0,Viettel,2,47435144,2019-11-11,HN,18_to_22,female
3,29080935,4174460303,2020-01-01,"10,000",13,0,Mobifone,3,29080935,2019-10-24,HN,18_to_22,female
4,14591075,4168216749,2020-01-01,"10,000",12,0,Viettel,2,14591075,2019-12-28,Other Cities,18_to_22,FEMALE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,54337200,8720390921,2020-12-31,"20,000",12,0,Viettel,2,54337200,2020-11-17,Other Cities,unknown,M
13491,59931360,8718580146,2020-12-31,"100,000",12,0,Viettel,2,59931360,2020-11-09,Other Cities,unknown,FEMALE
13492,46156488,8717515352,2020-12-31,"20,000",12,0,Viettel,2,46156488,2019-11-01,Other Cities,23_to_27,Nữ
13493,35858614,8716668316,2020-12-31,"10,000",12,0,Viettel,2,35858614,2019-11-04,HN,28_to_32,Nữ


In [451]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          13495 non-null  int64 
 1   order_id         13495 non-null  int64 
 2   Date             13495 non-null  object
 3   Amount           13495 non-null  object
 4   Merchant_id      13495 non-null  int64 
 5   Purchase_status  13495 non-null  int64 
 6   Merchant_name    13495 non-null  object
 7   Rate_pct         13495 non-null  int64 
 8   User_id          13495 non-null  int64 
 9   First_tran_date  13495 non-null  object
 10  Location         13495 non-null  object
 11  Age              13495 non-null  object
 12  Gender           13495 non-null  object
dtypes: int64(6), object(7)
memory usage: 1.3+ MB


## Clean data

In [452]:
df = merged_df

In [453]:
gender_mapping = {
    'FEMALE': 'Female',
    'Nữ': 'Female',
    'f': 'Female',
    'female': 'Female'
}

In [454]:
df['Gender'] = df['Gender'].map(lambda x: gender_mapping.get(x, 'Male'))
df

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,Merchant_name,Rate_pct,User_id,First_tran_date,Location,Age,Gender
0,21269588,4169517626,2020-01-01,"10,000",13,0,Mobifone,3,21269588,2018-12-11,HN,>37,Female
1,28097592,4170276686,2020-01-01,"20,000",13,0,Mobifone,3,28097592,2019-12-30,HN,>37,Female
2,47435144,4166729310,2020-01-01,"10,000",12,0,Viettel,2,47435144,2019-11-11,HN,18_to_22,Female
3,29080935,4174460303,2020-01-01,"10,000",13,0,Mobifone,3,29080935,2019-10-24,HN,18_to_22,Female
4,14591075,4168216749,2020-01-01,"10,000",12,0,Viettel,2,14591075,2019-12-28,Other Cities,18_to_22,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,54337200,8720390921,2020-12-31,"20,000",12,0,Viettel,2,54337200,2020-11-17,Other Cities,unknown,Male
13491,59931360,8718580146,2020-12-31,"100,000",12,0,Viettel,2,59931360,2020-11-09,Other Cities,unknown,Female
13492,46156488,8717515352,2020-12-31,"20,000",12,0,Viettel,2,46156488,2019-11-01,Other Cities,23_to_27,Female
13493,35858614,8716668316,2020-12-31,"10,000",12,0,Viettel,2,35858614,2019-11-04,HN,28_to_32,Female


#### Standardize Location

In [455]:
location_mapping = {
    'HCMC': 'Ho Chi Minh City',
    'Ho Chi Minh City': 'Ho Chi Minh City',
    'Other Cities': 'Other Cities',
    'Other': 'Other Cities',
    'HN': 'Ha Noi',
    'Unknown': 'Unknown',
}

In [456]:
df['Location'] = df['Location'].map(lambda x: location_mapping.get(x))
df

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,Merchant_name,Rate_pct,User_id,First_tran_date,Location,Age,Gender
0,21269588,4169517626,2020-01-01,"10,000",13,0,Mobifone,3,21269588,2018-12-11,Ha Noi,>37,Female
1,28097592,4170276686,2020-01-01,"20,000",13,0,Mobifone,3,28097592,2019-12-30,Ha Noi,>37,Female
2,47435144,4166729310,2020-01-01,"10,000",12,0,Viettel,2,47435144,2019-11-11,Ha Noi,18_to_22,Female
3,29080935,4174460303,2020-01-01,"10,000",13,0,Mobifone,3,29080935,2019-10-24,Ha Noi,18_to_22,Female
4,14591075,4168216749,2020-01-01,"10,000",12,0,Viettel,2,14591075,2019-12-28,Other Cities,18_to_22,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,54337200,8720390921,2020-12-31,"20,000",12,0,Viettel,2,54337200,2020-11-17,Other Cities,unknown,Male
13491,59931360,8718580146,2020-12-31,"100,000",12,0,Viettel,2,59931360,2020-11-09,Other Cities,unknown,Female
13492,46156488,8717515352,2020-12-31,"20,000",12,0,Viettel,2,46156488,2019-11-01,Other Cities,23_to_27,Female
13493,35858614,8716668316,2020-12-31,"10,000",12,0,Viettel,2,35858614,2019-11-04,Ha Noi,28_to_32,Female


### 1. Using data from the 'Commission' table, add a column 'Revenue' in the 'Transactions' table that displays MoMo's earned revenue for each order, and then calculate MoMo's total revenue in January 2020.

In [457]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          13495 non-null  int64 
 1   order_id         13495 non-null  int64 
 2   Date             13495 non-null  object
 3   Amount           13495 non-null  object
 4   Merchant_id      13495 non-null  int64 
 5   Purchase_status  13495 non-null  int64 
 6   Merchant_name    13495 non-null  object
 7   Rate_pct         13495 non-null  int64 
 8   User_id          13495 non-null  int64 
 9   First_tran_date  13495 non-null  object
 10  Location         13495 non-null  object
 11  Age              13495 non-null  object
 12  Gender           13495 non-null  object
dtypes: int64(6), object(7)
memory usage: 1.3+ MB


#### Standardize Date, First_tran_date

In [458]:
df.Date = df.Date.astype('datetime64[ns]')
df.First_tran_date = df.First_tran_date.astype('datetime64[ns]', errors='ignore')
df['First_tran_date'] = pd.to_datetime(df['First_tran_date'], errors='coerce')


In [459]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13495 non-null  int64         
 1   order_id         13495 non-null  int64         
 2   Date             13495 non-null  datetime64[ns]
 3   Amount           13495 non-null  object        
 4   Merchant_id      13495 non-null  int64         
 5   Purchase_status  13495 non-null  int64         
 6   Merchant_name    13495 non-null  object        
 7   Rate_pct         13495 non-null  int64         
 8   User_id          13495 non-null  int64         
 9   First_tran_date  13421 non-null  datetime64[ns]
 10  Location         13495 non-null  object        
 11  Age              13495 non-null  object        
 12  Gender           13495 non-null  object        
dtypes: datetime64[ns](2), int64(6), object(5)
memory usage: 1.3+ MB


In [460]:
df['Amount'] = df['Amount'].str.replace(',', '').astype(float)
df['Revenue'] = df['Rate_pct'] * df['Amount'] / 100

df['Date'] = pd.to_datetime(df['Date'])
df

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,Merchant_name,Rate_pct,User_id,First_tran_date,Location,Age,Gender,Revenue
0,21269588,4169517626,2020-01-01,10000.0,13,0,Mobifone,3,21269588,2018-12-11,Ha Noi,>37,Female,300.0
1,28097592,4170276686,2020-01-01,20000.0,13,0,Mobifone,3,28097592,2019-12-30,Ha Noi,>37,Female,600.0
2,47435144,4166729310,2020-01-01,10000.0,12,0,Viettel,2,47435144,2019-11-11,Ha Noi,18_to_22,Female,200.0
3,29080935,4174460303,2020-01-01,10000.0,13,0,Mobifone,3,29080935,2019-10-24,Ha Noi,18_to_22,Female,300.0
4,14591075,4168216749,2020-01-01,10000.0,12,0,Viettel,2,14591075,2019-12-28,Other Cities,18_to_22,Female,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,54337200,8720390921,2020-12-31,20000.0,12,0,Viettel,2,54337200,2020-11-17,Other Cities,unknown,Male,400.0
13491,59931360,8718580146,2020-12-31,100000.0,12,0,Viettel,2,59931360,2020-11-09,Other Cities,unknown,Female,2000.0
13492,46156488,8717515352,2020-12-31,20000.0,12,0,Viettel,2,46156488,2019-11-01,Other Cities,23_to_27,Female,400.0
13493,35858614,8716668316,2020-12-31,10000.0,12,0,Viettel,2,35858614,2019-11-04,Ha Noi,28_to_32,Female,200.0


In [461]:
january_data = df[df['Date'].dt.month == 1]
total_of_revenue_jan = january_data['Revenue'].sum()

In [462]:
print(f'Total revenue in January: {total_of_revenue_jan}')

Total revenue in January: 1409827.02


### 2. What is MoMo's most profitable month?

In [463]:
revenue_by_month = df.groupby(df['Date'].dt.month)['Revenue'].sum()
revenue_by_month = revenue_by_month.sort_values(ascending=False)
revenue_by_month

Date
9     1702200.00
10    1690900.00
12    1638200.00
8     1618700.00
6     1617200.00
3     1584000.00
7     1581900.00
11    1539300.00
4     1488300.00
5     1463700.00
1     1409827.02
2     1378500.00
Name: Revenue, dtype: float64

In [464]:
print(f'MoMo\'s most profitable month {revenue_by_month.idxmax()} with total revenue of {revenue_by_month.max()}')
print(f'MoMo\'s least profitable month {revenue_by_month.idxmin()} with total revenue of {revenue_by_month.min()}')

MoMo's most profitable month 9 with total revenue of 1702200.0
MoMo's least profitable month 2 with total revenue of 1378500.0


### 3. What day of the week does MoMo make the most money, on average? The least money?

In [465]:
df['Weekday'] = df['Date'].dt.day_name()
df

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,Merchant_name,Rate_pct,User_id,First_tran_date,Location,Age,Gender,Revenue,Weekday
0,21269588,4169517626,2020-01-01,10000.0,13,0,Mobifone,3,21269588,2018-12-11,Ha Noi,>37,Female,300.0,Wednesday
1,28097592,4170276686,2020-01-01,20000.0,13,0,Mobifone,3,28097592,2019-12-30,Ha Noi,>37,Female,600.0,Wednesday
2,47435144,4166729310,2020-01-01,10000.0,12,0,Viettel,2,47435144,2019-11-11,Ha Noi,18_to_22,Female,200.0,Wednesday
3,29080935,4174460303,2020-01-01,10000.0,13,0,Mobifone,3,29080935,2019-10-24,Ha Noi,18_to_22,Female,300.0,Wednesday
4,14591075,4168216749,2020-01-01,10000.0,12,0,Viettel,2,14591075,2019-12-28,Other Cities,18_to_22,Female,200.0,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,54337200,8720390921,2020-12-31,20000.0,12,0,Viettel,2,54337200,2020-11-17,Other Cities,unknown,Male,400.0,Thursday
13491,59931360,8718580146,2020-12-31,100000.0,12,0,Viettel,2,59931360,2020-11-09,Other Cities,unknown,Female,2000.0,Thursday
13492,46156488,8717515352,2020-12-31,20000.0,12,0,Viettel,2,46156488,2019-11-01,Other Cities,23_to_27,Female,400.0,Thursday
13493,35858614,8716668316,2020-12-31,10000.0,12,0,Viettel,2,35858614,2019-11-04,Ha Noi,28_to_32,Female,200.0,Thursday


In [466]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

weekday_counts = {}

for day in weekdays:
    count_unique_days = df[df['Weekday'] == day]['Date'].nunique()
    weekday_counts[day] = count_unique_days

total_revenue_weekday = df.groupby('Weekday')['Revenue'].sum().to_dict()

avg_revenue_by_weekday = {day: total_revenue_weekday[day] / weekday_counts[day] for day in weekdays}

# sort the dictionary by value
sorted_avg_revenue_by_weekday = dict(sorted(avg_revenue_by_weekday.items(), key=lambda item: item[1], reverse=True))
sorted_avg_revenue_by_weekday = pd.DataFrame(sorted_avg_revenue_by_weekday.items(), columns=['Weekday', 'Average Revenue'])
sorted_avg_revenue_by_weekday



,Weekday,Average Revenue
0,Wednesday,57818.867925
1,Friday,54405.769231
2,Thursday,52107.547170
3,Tuesday,51580.769231
4,Saturday,48719.230769
5,Monday,48615.384615
6,Sunday,44498.596538


In [467]:
print(f'The most profitable day of the week is {sorted_avg_revenue_by_weekday.iloc[0, 0]} with an average revenue of {sorted_avg_revenue_by_weekday.iloc[0, 1]}')
print(f'The least profitable day of the week is {sorted_avg_revenue_by_weekday.iloc[-1, 0]} with an average revenue of {sorted_avg_revenue_by_weekday.iloc[-1, 1]}')

The most profitable day of the week is Wednesday with an average revenue of 57818.8679245283
The least profitable day of the week is Sunday with an average revenue of 44498.59653846154


### 4. Combined with the 'User_Info' table, add columns: Age, Gender, Location, Type_user (New/Current) in 'Transactions' table and calculate the total number of new users in December 2020.

In [468]:


df['Type_user'] = df.apply(lambda row: 'New' if row['First_tran_date'].month == row['Date'].month and row['First_tran_date'].year == row['Date'].year else 'Current', axis=1)
new_user_count = df[(df["Type_user"] == "New") & (df["Date"].dt.month == 12)]["user_id"].nunique()
print(f'The number of new users in December: {new_user_count}')

The number of new users in December: 72


In [469]:
# save the final DataFrame to a CSV file wwith utf-8 encoding
df.to_csv('cleaned_data.csv', index=False, encoding='utf-8')

1. xu hướng độ tuổi sử dụng dịch vụ này (chiến lược marketing hướng tới)
2. mệnh giá người dùng thường mua (chiến lược giảm giá)
3. Gender